# __Diffusion Tensor Imaging (DTI) analysis notebook__
#### __Last updated on:__ 27/02/2020
#### __Author:__ Rakshit Dadarwal

### __Requirements:__
#### 1. Single or multi-shell DWI data
#### 2. DIPY (https://dipy.org/)

### __This script includes:__
#### 1. DTI parametric maps estimation
#### 2. Compute color-coded or RGB Fractional Anisotropy (FA) map 

### __Import python libraries__ 

In [ ]:
import os
import nibabel as nib
import timeit; timeit.timeit()

from dipy.io import read_bvals_bvecs
from dipy.core.gradients import gradient_table
from dipy.reconst.dti import TensorModel
from dipy.reconst.dti import fractional_anisotropy
from dipy.reconst.dti import color_fa

### __Define DWI data path__

In [ ]:
#--------------------------------------------------
# Pre-processed DWI data and brain mask
#--------------------------------------------------
data_path = "/home/User/Diffusion"
dwi_file = 'eddy_corrected.nii.gz'
brainmask_file = 'brainmask.nii.gz'
bval = 'dMRI.bvals'
bvec = 'eddy_corrected.eddy_rotated_bvecs'

### __Post-processing__

In [ ]:
#--------------------------------------------------
# load DWI data files
#--------------------------------------------------
img1 = nib.load(os.path.join(data_path,dwi_file))
data = img1.get_data()

img2 = nib.load(os.path.join(data_path,brainmask_file))
brainmask = img2.get_data()

bvals, bvecs = read_bvals_bvecs(os.path.join(bval),
                                os.path.join(data_path,bvec))
gtab = gradient_table(bvals, bvecs)

#--------------------------------------------------------------
#               Fit diffusion tensor model
#--------------------------------------------------------------
print('Fitting diffuison tensor model')

ten_model = TensorModel(gtab)
ten_fit = ten_model.fit(data, brainmask)
        
#--------------------------------------------------------------
#               Save DTI parametric maps
#--------------------------------------------------------------
if not os.path.exists(data_path+'/DTI/'):
    os.mkdir(data_path+'/DTI')
output_path = data_path+'/DTI/'
        
FA = ten_fit.fa
AD = ten_fit.ad
RD = ten_fit.rd
MD = ten_fit.md
        
nib.save(nib.Nifti1Image(FA, img1.affine), os.path.join(output_path,'FA.nii.gz'))
nib.save(nib.Nifti1Image(MD, img1.affine), os.path.join(output_path,'MD.nii.gz'))
nib.save(nib.Nifti1Image(RD, img1.affine), os.path.join(output_path,'RD.nii.gz'))
nib.save(nib.Nifti1Image(AD, img1.affine), os.path.join(output_path,'AD.nii.gz'))
    
#--------------------------------------------------------------
#               Save FA RGB map
#--------------------------------------------------------------
fa = fractional_anisotropy(ten_fit.evals)
cfa = color_fa(fa, ten_fit.evecs)
FA = np.clip(fa, 0, 1)
RGB = color_fa(fa, ten_fit.evecs)

nib.save(nib.Nifti1Image(np.array(255 * cfa, 'uint8'), img1.affine), os.path.join(output_path,'FA_RGB.nii.gz'))

print('Elapsed time:',timeit.timeit())
print('Enjoy!!')